In [849]:
import os, shutil
import pandas as pd
import os
import re
import string

import numpy as np
import matplotlib.pyplot as plt
from cycler import cycler
import statistics
from scipy.stats.mstats import gmean

from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [900]:
import os
cwd = os.getcwd()

In [902]:
gen=pd.read_csv(cwd+"//data//Generator.csv")
gen_prop=pd.read_csv(cwd+"//data//Generator_proposed.csv")
PP=pd.read_csv(cwd+"//data//PP_loc.csv")


In [851]:
gen.columns

Index(['Utility ID', 'Utility Name', 'Plant Code', 'Plant Name', 'State',
       'County', 'Generator ID', 'Technology', 'Prime Mover', 'Unit Code',
       'Ownership', 'Duct Burners',
       'Can Bypass Heat Recovery Steam Generator?',
       'RTO/ISO LMP Node Designation',
       'RTO/ISO Location Designation for Reporting Wholesale Sales Data to FERC',
       'Nameplate Capacity (MW)', 'Nameplate Power Factor',
       'Summer Capacity (MW)', 'Winter Capacity (MW)', 'Minimum Load (MW)',
       'Uprate or Derate Completed During Year',
       'Month Uprate or Derate Completed', 'Year Uprate or Derate Completed',
       'Status', 'Synchronized to Transmission Grid', 'Operating Month',
       'Operating Year', 'Planned Retirement Month', 'Planned Retirement Year',
       'Associated with Combined Heat and Power System', 'Sector Name',
       'Sector', 'Topping or Bottoming', 'Energy Source 1', 'Energy Source 2',
       'Energy Source 3', 'Energy Source 4', 'Energy Source 5',
       'Ene

In [852]:
gen.shape



(24609, 73)

In [853]:
gen_prop.shape

(1554, 47)

In [854]:
Gen_tot =  pd.concat([gen, gen_prop], ignore_index=True)

In [855]:

Gen_tot =Gen_tot.drop_duplicates(subset=['Plant Code', 'Generator ID', ], keep='last')

In [856]:

Gen_tot.shape

(26161, 78)

In [857]:
PP.columns
PP = PP[['Plant Code', 'Plant Name', 'Latitude',
       'Longitude']]

In [878]:
PP_merged = pd.merge(Gen_tot, PP, on = ['Plant Name','Plant Code'])

PP_merged

,Utility ID,Utility Name,Plant Code,Plant Name,State,County,Generator ID,Technology,Prime Mover,Unit Code,...,Multiple Fuels?,Cofire Fuels?,Switch Between Oil and Natural Gas?,Effective Month,Effective Year,Current Month,Current Year,Previously Canceled,Latitude,Longitude
0,63560,"TDX Sand Point Generating, LLC",1.0,Sand Point,AK,Aleutians East,1,Petroleum Liquids,IC,NaN,...,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.339722,-160.497222
1,63560,"TDX Sand Point Generating, LLC",1.0,Sand Point,AK,Aleutians East,2,Petroleum Liquids,IC,NaN,...,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.339722,-160.497222
2,63560,"TDX Sand Point Generating, LLC",1.0,Sand Point,AK,Aleutians East,3,Petroleum Liquids,IC,NaN,...,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.339722,-160.497222
3,63560,"TDX Sand Point Generating, LLC",1.0,Sand Point,AK,Aleutians East,5,Petroleum Liquids,IC,NaN,...,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.339722,-160.497222
4,63560,"TDX Sand Point Generating, LLC",1.0,Sand Point,AK,Aleutians East,WT1,Onshore Wind Turbine,WT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.339722,-160.497222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26155,64913,"SR Paris, LLC",65641.0,SR Paris,TN,Henry,SRPAR,Solar Photovoltaic,PV,NaN,...,N,NaN,NaN,10,2022,10,2022,N,36.445894,-88.327688
26156,64914,"SR Jackson II, LLC",65642.0,SR Jackson II,TN,Madison,SRJA2,Solar Photovoltaic,PV,NaN,...,N,NaN,NaN,8,2022,8,2022,N,35.607757,-88.917046
26157,64927,"IP Lumina II, LLC",65644.0,Lumina II Solar Project,TX,Scurry,LUMII,Solar Photovoltaic,PV,NaN,...,N,NaN,NaN,6,2023,6,2023,N,32.883528,-100.746389
26158,64924,"IP Lumina, LLC",65645.0,Lumina Solar Project,TX,Scurry,LUMIN,Solar Photovoltaic,PV,NaN,...,N,NaN,NaN,6,2023,6,2023,N,32.901081,-100.982695


In [879]:
PP_of_interest = 'Nuclear|Natural Gas|Coal|Onshore Wind|Solar Photovoltaic|Hydro'

Plants = PP_merged['Technology'].str.contains(PP_of_interest, case=False, na=False)
PP_merged[Plants]=PP_merged[Plants][PP_merged[Plants]['Technology'].str.contains("Other|Storage| Gasification ") == False]
PP_merged[Plants].shape

(19079, 80)

In [880]:
PP_merged[Plants].columns
PP_merged= PP_merged[PP_merged['Status'].str.contains("RE|SE|SB")==False]

In [881]:
df2 = PP_merged[Plants][[ 'Plant Code', 'Generator ID', 'Plant Name', 'Latitude', 'Longitude','Technology','Nameplate Capacity (MW)', 
       'Operating Month','Operating Year', 'Planned Retirement Month', 'Planned Retirement Year',
     'Current Month', 'Current Year']]
df2=df2[df2['Technology'].notna()]
df2.columns = ['Plant Code', 'Generator ID', 'name', 'Latitude', 'Longitude','source','capacity', 
       'Operating Month','Operating Year', 'Planned Retirement Month', 'Planned Retirement Year',
     'Current Month', 'Current Year']





<ipython-input-881-c57ec0d6aaec>:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [882]:
df2['Operating Year'].fillna(df2['Current Year'], inplace=True)
del df2['Current Year']
df2['Operating Month'].fillna(df2['Current Month'], inplace=True)
del df2['Current Month']


In [883]:
df2=df2.assign(coordinate= "("+df2['Latitude'].astype(str)+','+df2['Longitude'].astype(str)+")")
del df2['Latitude']
del df2['Longitude']

In [884]:
df2 = df2.replace(' ',np.nan)
df2['Planned Retirement Month'].fillna(df2['Operating Month'], inplace=True)
df2

,Plant Code,Generator ID,name,source,capacity,Operating Month,Operating Year,Planned Retirement Month,Planned Retirement Year,coordinate
4,1.0,WT1,Sand Point,Onshore Wind Turbine,0.5,10.0,2011.0,10.0,NaN,"(55.339722,-160.497222)"
5,1.0,WT2,Sand Point,Onshore Wind Turbine,0.5,10.0,2011.0,10.0,NaN,"(55.339722,-160.497222)"
6,2.0,1,Bankhead Dam,Conventional Hydroelectric,53.9,7.0,1963.0,7.0,NaN,"(33.458665,-87.356823)"
7,3.0,1,Barry,Natural Gas Steam Turbine,153.1,2.0,1954.0,2.0,NaN,"(31.0069,-88.0103)"
8,3.0,2,Barry,Natural Gas Steam Turbine,153.1,7.0,1954.0,7.0,NaN,"(31.0069,-88.0103)"
...,...,...,...,...,...,...,...,...,...,...
26155,65641.0,SRPAR,SR Paris,Solar Photovoltaic,6.8,10,2022,10,NaN,"(36.445894,-88.327688)"
26156,65642.0,SRJA2,SR Jackson II,Solar Photovoltaic,1,8,2022,8,NaN,"(35.607757,-88.917046)"
26157,65644.0,LUMII,Lumina II Solar Project,Solar Photovoltaic,357.8,6,2023,6,NaN,"(32.883528,-100.746389)"
26158,65645.0,LUMIN,Lumina Solar Project,Solar Photovoltaic,357.8,6,2023,6,NaN,"(32.901081,-100.982695)"


In [885]:
df2.loc[df2['source'].str.contains('Gas'), 'source'] = 'gas'
df2.loc[df2['source'].str.contains('Coal'), 'source'] = 'coal'
df2.loc[df2['source'].str.contains('Wind'), 'source'] = 'wind'
df2.loc[df2['source'].str.contains('Solar'), 'source'] = 'solar'
df2.loc[df2['source'].str.contains('Hydro'), 'source'] = 'hydro'
df2.loc[df2['source'].str.contains('Nuclear'), 'source'] = 'nuclear'

In [886]:
df2=df2.assign(name= df2['name'].astype(str)+'_'+df2['Generator ID'].astype(str))
del df2['Generator ID']


In [887]:
df2.name = df2.name.str.replace(' ','_')
df2

,Plant Code,name,source,capacity,Operating Month,Operating Year,Planned Retirement Month,Planned Retirement Year,coordinate
4,1.0,Sand_Point_WT1,wind,0.5,10.0,2011.0,10.0,NaN,"(55.339722,-160.497222)"
5,1.0,Sand_Point_WT2,wind,0.5,10.0,2011.0,10.0,NaN,"(55.339722,-160.497222)"
6,2.0,Bankhead_Dam_1,hydro,53.9,7.0,1963.0,7.0,NaN,"(33.458665,-87.356823)"
7,3.0,Barry_1,gas,153.1,2.0,1954.0,2.0,NaN,"(31.0069,-88.0103)"
8,3.0,Barry_2,gas,153.1,7.0,1954.0,7.0,NaN,"(31.0069,-88.0103)"
...,...,...,...,...,...,...,...,...,...
26155,65641.0,SR_Paris_SRPAR,solar,6.8,10,2022,10,NaN,"(36.445894,-88.327688)"
26156,65642.0,SR_Jackson_II_SRJA2,solar,1,8,2022,8,NaN,"(35.607757,-88.917046)"
26157,65644.0,Lumina_II_Solar_Project_LUMII,solar,357.8,6,2023,6,NaN,"(32.883528,-100.746389)"
26158,65645.0,Lumina_Solar_Project_LUMIN,solar,357.8,6,2023,6,NaN,"(32.901081,-100.982695)"


In [888]:
def my_func(row):
    if row['source'] == 'nuclear':
        val = 60.0
    elif row['source']  == 'solar':
        val = 25.0
    elif row['source']  == 'wind':
        val = 20.0
    elif row['source']  == 'coal':
        val = 30.0        
    elif row['source']  == 'gas':
        val = 30.0          
    else:
        val = 40.0
    return val

df2['plant_life'] = df2.apply(my_func, axis=1)
df2

,Plant Code,name,source,capacity,Operating Month,Operating Year,Planned Retirement Month,Planned Retirement Year,coordinate,plant_life
4,1.0,Sand_Point_WT1,wind,0.5,10.0,2011.0,10.0,NaN,"(55.339722,-160.497222)",20.0
5,1.0,Sand_Point_WT2,wind,0.5,10.0,2011.0,10.0,NaN,"(55.339722,-160.497222)",20.0
6,2.0,Bankhead_Dam_1,hydro,53.9,7.0,1963.0,7.0,NaN,"(33.458665,-87.356823)",40.0
7,3.0,Barry_1,gas,153.1,2.0,1954.0,2.0,NaN,"(31.0069,-88.0103)",30.0
8,3.0,Barry_2,gas,153.1,7.0,1954.0,7.0,NaN,"(31.0069,-88.0103)",30.0
...,...,...,...,...,...,...,...,...,...,...
26155,65641.0,SR_Paris_SRPAR,solar,6.8,10,2022,10,NaN,"(36.445894,-88.327688)",25.0
26156,65642.0,SR_Jackson_II_SRJA2,solar,1,8,2022,8,NaN,"(35.607757,-88.917046)",25.0
26157,65644.0,Lumina_II_Solar_Project_LUMII,solar,357.8,6,2023,6,NaN,"(32.883528,-100.746389)",25.0
26158,65645.0,Lumina_Solar_Project_LUMIN,solar,357.8,6,2023,6,NaN,"(32.901081,-100.982695)",25.0


In [889]:
df2['Operating Year'] = df2['Operating Year'].astype(float)
df2['plant_life'] = df2['plant_life'].astype(float)

df2['Retirement'] = df2['Operating Year'] +df2['plant_life']

In [890]:
df2['Planned Retirement Year'].fillna(df2['Retirement'], inplace=True)
del df2['Retirement']
df2

,Plant Code,name,source,capacity,Operating Month,Operating Year,Planned Retirement Month,Planned Retirement Year,coordinate,plant_life
4,1.0,Sand_Point_WT1,wind,0.5,10.0,2011.0,10.0,2031.0,"(55.339722,-160.497222)",20.0
5,1.0,Sand_Point_WT2,wind,0.5,10.0,2011.0,10.0,2031.0,"(55.339722,-160.497222)",20.0
6,2.0,Bankhead_Dam_1,hydro,53.9,7.0,1963.0,7.0,2003.0,"(33.458665,-87.356823)",40.0
7,3.0,Barry_1,gas,153.1,2.0,1954.0,2.0,1984.0,"(31.0069,-88.0103)",30.0
8,3.0,Barry_2,gas,153.1,7.0,1954.0,7.0,1984.0,"(31.0069,-88.0103)",30.0
...,...,...,...,...,...,...,...,...,...,...
26155,65641.0,SR_Paris_SRPAR,solar,6.8,10,2022.0,10,2047.0,"(36.445894,-88.327688)",25.0
26156,65642.0,SR_Jackson_II_SRJA2,solar,1,8,2022.0,8,2047.0,"(35.607757,-88.917046)",25.0
26157,65644.0,Lumina_II_Solar_Project_LUMII,solar,357.8,6,2023.0,6,2048.0,"(32.883528,-100.746389)",25.0
26158,65645.0,Lumina_Solar_Project_LUMIN,solar,357.8,6,2023.0,6,2048.0,"(32.901081,-100.982695)",25.0


In [891]:
df2['Planned Retirement Year'] = df2['Planned Retirement Year'].astype(int)
plants = df2.loc[df2['Planned Retirement Year'] > 2019] 

In [892]:
plants 

,Plant Code,name,source,capacity,Operating Month,Operating Year,Planned Retirement Month,Planned Retirement Year,coordinate,plant_life
4,1.0,Sand_Point_WT1,wind,0.5,10.0,2011.0,10.0,2031,"(55.339722,-160.497222)",20.0
5,1.0,Sand_Point_WT2,wind,0.5,10.0,2011.0,10.0,2031,"(55.339722,-160.497222)",20.0
11,3.0,Barry_A1CT,gas,170.1,5.0,2000.0,5.0,2030,"(31.0069,-88.0103)",30.0
12,3.0,Barry_A1CT2,gas,170.1,5.0,2000.0,5.0,2030,"(31.0069,-88.0103)",30.0
13,3.0,Barry_A1ST,gas,195.2,5.0,2000.0,5.0,2030,"(31.0069,-88.0103)",30.0
...,...,...,...,...,...,...,...,...,...,...
26155,65641.0,SR_Paris_SRPAR,solar,6.8,10,2022.0,10,2047,"(36.445894,-88.327688)",25.0
26156,65642.0,SR_Jackson_II_SRJA2,solar,1,8,2022.0,8,2047,"(35.607757,-88.917046)",25.0
26157,65644.0,Lumina_II_Solar_Project_LUMII,solar,357.8,6,2023.0,6,2048,"(32.883528,-100.746389)",25.0
26158,65645.0,Lumina_Solar_Project_LUMIN,solar,357.8,6,2023.0,6,2048,"(32.901081,-100.982695)",25.0


In [893]:
plants.to_csv("//data//PPlants2.csv")